<h1><b><center>Mecánica Celeste</center></b></h1>
<h2><b><center>Prof. Jorge I. Zuluaga</center></b></h1>
<h2><b><center>Tarea 4</center></b><h2>
<h3><b><center>Viaje a la Luna</center></b><h3>
<h5><center><b>Asignada</b>: 12 de Octubre de 2023</center><h5>
<h5><center><b>Entrega</b>: 26 de Octubre de 2023</center><h5>
</p>

<hr/>
<b>Nombre</b>: Sebastián Numpaque
<br/>
<b>Cédula</b>: 1002396960
<br/>
<b>Última actualización</b>: 2023-10-25
<hr/>

## Enunciado

**Objetivo**: En esta tarea nos proponemos estudiar las denominadas trayectorias de retorno libre a la Luna usando la teoría del CRTBP, apoyado por lo que sabemos el problema de los dos cuerpos.

**Procedimiento**:

Responda primero estas cuestiones usando el problema de los dos cuerpos.

- Una nave espacial se encuentra en una órbita circular inicial alrededor de la Tierra a una distancia de 36 000 km del centro de nuestro planeta. Calcule la velocidad que debe tener la nave respecto al centro de la Tierra para que se mantenga en una órbita circular con ese radio.

- ¿Cuánto debe aumentar la velocidad, $\Delta v$, para que la nave pase de estar en una órbita circular alrededor de la Tierra a una órbita elíptica que tenga perigeo igual a 36 000 km y apogeo igual a la distancia promedio de la Luna (383 000 km)?. A está orbita se la llama *orbita de transferencia de Hohmann*. Obviamente está orbita asume que no existe la Luna.

- Suponga que la Luna arranca en t = 0, en x = 383 000 km, con una órbita circular alrededor de la Tierra. ¿En qué anomalía verdadera debe recibir la nave el impulso $\Delta v$ para que cuando llegue al apogeo la Luna se encuentre con la nave?. Esta es la condición básica que define el caso de una transferencia de Hohmann.

- Calcule las componentes cartesianas de la posición y la velocidad inicial de la nave respecto de la Tierra en la órbita de Hohmann.

Con estos valores indicativos vamos ahora a simular el sistema Tierra-Luna-Nave y a estudiar la trayectoria del vehículo, tanto en un sistema de referencia inercial, como en un sistema de referencia rotante en el que la Tierra y la Luna están en reposo (sistema del CRTBP).

1. Calcule la posición y velocidad inicial de la Tierra en un sistema de referencia inercial en el que el origen está en el centro de masa de la Tierra y la Luna y la Luna y la Tierra orbitan con un semieje mayor relativo igual a 383 000 km.

2. Calcule ahora la posición y velocidad inicial de la nave en el sistema de referencia inercial y en el momento en el que recibe el impulso Para ello simplemente sume a la posición y velocidad inicial de la nave, la posición y velocidad inicial de la Tierra en t = 0.

3. Simule el movimiento de la nave usando la rutina `crtbp_solucion` y mire cómo es la trayectoria, tanto en el sistema rotante como en el sistema de referencia inercial. Calcule la constante de Jacobi, grafique las regiones de exclusión y la superficie de cero velocidad correspondiente a este valor de la constante de Jacobi.

4. Es posible que en el primer intento no consiga la trayectoria deseada (una trayectoria que lleve la nave hasta la Luna y que después la regrese a la Tierra). Modifique con cuidado las condiciones iniciales de la nave (en especial la velocidad incial) hasta que consiga una o varias trayectorias de retorno libre en el sistema. Calcule el apogeo y el perigeo de la trayectoria de la nave respecto a la Tierra y el tiempo que tardaría la nave en salir de la órbita alrededor de la Tierra y en regresar a un punto cercano.

**Resultados esperados**:

- Este notebook con el procedimiento debidamente comentado.

- Datos mínimos:
  - Velocidad de la nave en la órbita circular circumterrestre.
  - Valor del impulso $\Delta v$ para la transferencia de Hohmann.
  - Anomalía inicial de la nave para llegar a la Luna.

- Gráficos mínimos:
  - Gráfico de una o varias trayectorias de retorno libre de la Tierra a la Luna, que incluyan, regiones de exclusión, superficies de cero velocidad-

- Gráficos opcionales:
  - Animación de las trayectorias de retorno libre.

- En caso de utilizar ayudas como `ChatGPT` o códigos de `Stack Over Flow` debe citar debidamente la fuente, pero también entender a cabalidad el código, el profesor puede pedir justificación.



---



#**Cargue e Importe de Librerias y Módulos**

In [1]:
# @title
!pip install -q pymcel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.9/888.9 kB 50.2 MB/s eta 0:00:00


In [2]:
# @title
#Comunes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Astronómicas
import pymcel as pc
import spiceypy as spy
import astropy.constants as c

#Gráficos
import plotly.graph_objects as go

#Funciones de Raices e Interpolación
from scipy.optimize import ridder
from scipy.interpolate import CubicHermiteSpline

#**Unidades Canónicas**

Se toman las unidades canónicas del **CRTBP**

$$[U_L]=d_{Earth-Moon}\hspace{1cm}[U_M]=M_{Earth} + M_{Moon} \hspace{1cm}[U_T]\approx4.32\,d$$

In [3]:
# @title
#Kernel de Spice
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de440.tpc

--2023-10-26 17:30:42--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de440.tpc
Resolving naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connecting to naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12708 (12K) [text/plain]
Saving to: ‘gm_de440.tpc’

gm_de440.tpc        100%[===================>]  12.41K  --.-KB/s    in 0s      

2023-10-26 17:30:42 (100 MB/s) - ‘gm_de440.tpc’ saved [12708/12708]



In [20]:
# @title
#Obtencion de Valores de GM
G = c.G.value
spy.furnsh('gm_de440.tpc')
M_earth = (spy.bodvrd("399","GM",1)[1][0])*1e9/G
M_moon = (spy.bodvrd("301","GM",1)[1][0])*1e9/G

#Unidades del CRTBP
UM = M_earth + M_moon
UL = 3.83*1e8
UT = np.sqrt(UL**3/(G*UM))
UV = UL/UT

#Frecuencia angular Tierra - Luna
omega = 1
omega_vec = np.array([0, 0 , omega])

#Parametros Gravitacionales
mu2 = M_moon/UM
mu1 = 1 - mu2
mu = 1
alpha = mu2.copy()



---



#**Two Body Problem**

- **1.** En una órbita circular, la velocidad requerida para mantener la trayectoria a una distancia $R_1$ de la *Tierra* se relaciona con la aceleración gravitacional que experimenta la nave espacial.

  $$a_c=\frac{\mu_1}{R_1^2}=\frac{v_c^2}{R_1}\\ v_c=\sqrt{\frac{\mu_1}{R_1}}$$

- **2.** Si se desea realizar una órbita de transferencia de *Hohmann*, esta nueva trayectoria elíptica será cofocal con la órbita circular inicial. Convenientemente, el perigeo en la órbita de *Hohmann* coincide con cualquiera de los puntos sobre la trayectoria circular ($q=R_1$); su velocidad en este punto se obtiene a través de la *Vis Viva*

  $$v^2=\mu_1\left(\frac{2}{r}-\frac{1}{a}\right)$$

  En el perigeo se tiene $r=q=a(1-e)$, de modo que
  
  $$a=\frac{q}{1-e}=\frac{R_1}{1-e}$$
  
  La solución a la velocidad en este punto resulta ser

  $$v_q=\sqrt{\frac{\mu_1(1+e)}{R_1}}=v_c\sqrt{1+e}$$

  El punto de apogeo al que se desea llegar con la órbita de trasnferencia es un punto sobre la órbita "circular" de la *Luna*, es decir $Q=R_2$.
  
  $$Q=a(1+e)$$
  
  El *semi-eje mayor* de la órbita de transferencia se observa que es igual a

  $$a=\frac{R_1 + R_2}{2}$$

  De este modo, el factor $(1+e)$ resulta

  $$1+e=\frac{Q}{a}=\frac{2R_2}{R_1+R_2}$$

  Con estas expresiones, el valor de $v_q$ se reescribe de la siguiente forma

  $$v_q = v_c\sqrt{\frac{2R_2}{R_1+R_2}}$$

  Puesto que con $v_c$ se mantiene una trayectoria circular, para que la nave se ajuste a la órbita de transferencia requiere tener una velocidad igual a $v_q$. La cantidad de velocidad adicional que requiere para adaptarse a la nueva trayectoria es entonces

  $$\Delta v=v_q-v_c=v_c\left(\sqrt{\frac{2R_2}{R_1+R_2}}-1\right)$$

- **3.** Planteemos un primer sistema de referencia coplanar entre la *Tierra*, la *Luna* con origen en el centro de la *Tierra* y cuyo eje $\hat{x}$ se encuentra alineado el perigeo de la órbita de *Hohmann*.

  Cuando la nave inicia su transferencia, en el perigeo, se tiene una anomalía verdadera $f_0=0$ y cuando llega al apogeo e intercepta la órbita lunar tiene una $f=\pi$; este valor de $f$ es el mismo que tiene la Luna en este punto de intercepto.

  Mediante la *Tercera Ley de Kepler*, el periodo órbital de la nave en la trayectoria de *Hohmann* es

  $$T=2\pi\sqrt{\frac{a^3}{\mu_1}}=\pi\sqrt{\frac{(R_1+R_2)^3}{2\mu_1}}$$

  Desde que la nave inicia la transferencia hasta que intercepta a la *Luna*, ha recorrido media trayectoria, por lo que el tiempo transcurrido es $t=T/2$. Planteamos la siguiente expresión para la anomalía verdadera de la *Luna*, considerando su órbita circular

  $$f_{L}=f_{0L}+\omega_L t$$

  La rapidez angular a la que se mueve la *Luna* es simplemente

  $$\omega_L = \frac{v_L}{R_2}=\sqrt{\frac{\mu}{R_2^3}}$$

  Como mencionamos, cuando $t=T/2$, la *Luna* se encuentra en $f_L=\pi$ por tanto

  $$f_{0L}=\pi-\frac{\omega_L T}{2}=\pi\left(1-\sqrt{\frac{\mu(R_1+R_2)^3}{8\mu_1R_2^3}}\right)$$

  Si realizamos una rotación del eje $\hat{x}$ de modo que concuerde con $f_{0L}$, es decir, desde donde parte la *Luna* cuando la nave inicia la transferencia, entonces el valor de $f_0$ para la nave resulta ser el negativo de $f_{0L}$

  $$f_0=-f_{0L}=\pi\left(\sqrt{\frac{\mu(R_1+R_2)^3}{8\mu_1R_2^3}}-1\right)$$

- **4.** Cuando la nave recibe el impulso de transferencia, el punto de partida aún se encuentra sobre la órbita circular por lo que la posición inicial resulta simplemente

  $$\vec{r_0}=R_1(\cos{f_0}\hat{i}+\sin{f_0}\hat{j})$$

  En este instante que corresponde al momento donde la nave se encuentra en su perigeo, la velocidad es igual a $v_q$. Convenientemente, este punto se encuentra sobre la órbita circular inicial, por tanto se puede pensar en que la nave realice una órbita prograda o retrograda; en todo caso se tiene respectivamente.

  $$\vec{v_0} = v_q(\mp \sin{f_0}\hat{i}\pm\cos{f_0}\hat{j})$$



Se establecen los parametros para los calculos de la órbita de Transferencia de Hohmann

In [26]:
# @title
#Radios de los cuerpos
R_earth = 6.371e6/UL
R_moon = 1.7374e6/UL

#Altitud en Perigeo y Apogeo respecto a los Cuerpos
hq = 1.7e5/UL
hQ = 2e5/UL

#Radios Orbitas Circulares
R1 = R_earth + hq
R2 = 1 + R_moon + hQ

#Semi-major axis, Hohmann Transfer Orbit
a = (R1 + R2)/2

#Velocidad circular de la nave
vc = np.sqrt(mu1/R1)

#Velocidad en Perigeo de Orbita de Hohmann
vq = vc*np.sqrt(R2/a)

#Impulso
delta_v = vq - vc

#Periodo de la nave, Orbita de Hohmann
T_hohmann = 2*np.pi*np.sqrt((a**3)/mu1)

#Anomalia inicial
f0 = omega*T_hohmann/2 - np.pi

pd.DataFrame([vc*UV, delta_v*UV, np.rad2deg(f0), T_hohmann*UT/86400],
             index = ['Vc (m/s)', 'ΔV (m/s)', 'f0 (deg)', 'T Hohmann (day)'],
             columns = ['Algunos Valores'])

,Algunos Valores
Vc (m/s),7806.328264
ΔV (m/s),3140.869396
f0 (deg),-113.832354
T Hohmann (day),9.975024




---



#**CRTBP - Free Return Trajectory**

Aunque la órbita de transferencia de *Hohmann* resulta apropiada en problemas de *2Cuerpos*, cuando pasamos a un sistema como el *CRTBP*, esta solamente nos permite tener una idea del orden y las magnitudes de las condiciones iniciales de partida.

- **Sobre las Orbitas Circulares**

  **1.** La primera corrección que realizamos es la de pretender llegar a órbita lunar con cierta altitud respecto a la superficie lunar ($h_Q=200\, km$), es decir, que la órbita superior de la transferencia no coincide con el centro de masa lunar sino que es una órbita ligeramente superior. Con esto, ingenuamente, garantizaríamos que no estamos planeando una misión con destino de colisión con la Luna y disminuimos levemente la influencia gravitacional de la Luna (sigue siendo muy considerable, pero a mayor altitud es menor la atracción). Se toma $R_2$ como el radio de la órbita circular superior como la distancia *Tierra - Luna* más el radio lunar junto la altitud

  $$R_2\,[U_L] = 1 + R_{Moon}\,[U_L] + h_Q\,[U_L]$$

  **2.** Luego de varios intentos fallidos por lograr una orbita de retorno libre simétrica, y de considerar un análisis realista, se decidió disminuir el radio de la orbita circular de partida. En un contexto real, partir de una órbita de radio *36 000 Km* respecto al centro de la Tierra es llevar una nave hasta una altura de *~30 000 km* de la superficie terrestre lo que resulta costoso.

  Se estableció una órbita circular de partida con una altitud menor ($h_q=170\,km$). De este modo, el valor de $R_1$ resulta, respecto al centro de la Tierra, como el radio terrestre más la altitud sobre la superficie.

  $$R_1\,[U_L]=R_{Earth}\,[U_L]+h_q\,[U_L]$$

- **Sobre la Orientación**

  El sentido común haría pensar que la órbita que describe la nave es únicamente en sentido progrado, o con movimiento en la dirección de rotación de los cuerpos principales; sin embargo, con los intentos de construcción de la órbita de retorno libre se consideró la opción de realizar un viaje retrogrado. Este cambio en el sentido de movimiento de la nave permitió construir una órbita menos caótica y mucho más simétrica.

- **Sobre las Velocidades**

  Inicialmente se estableció que se debía adicionar la velocidad inercial de la Tierra a la nave. Debido a que el sentido de la órbita se realizó de forma retrograda, esta adición no se realizó pues le habría restado impulso a la nave para lograr el objetivo.

- **Sobre la Anomalia de Partida**

  El calculo que se realizó de la anomalía inicial para la transferencia de Hohmann ($f_0\simeq247°$) resultó en un valor mayor al que convenía. Con las númerosas ejecuciones del programa, se observó que la anomalía adecuada para la partida ($f_0=227.5$) era menor a la calculada. Esto favoreció la construcción de la órbita pues, considerando el sentido retrogrado, una anomalia menor aporta un mejor posicionamiento y contribuye al impulso de la nave al aprovechar mejor la dirección de velocidad.

- **Sobre el Apogeo, Perigeo y el Tiempo de Vuelo**

  La orbita se ejecutó en un tiempo igual al periodo que se calculó en la aproximación de la orbita de Hohmann ($T\simeq10\,d$). El tiempo de vuelo que resultó en el sistema del *CRTBP* ($T\simeq 7.5\,d$) resultó en aproximadamente *1/4* más corto al tiempo de la orbita de Hohmann.

  Se observa la simetría de la órbita y por tanto, se pudo simplificar el calculo del Perigeo y el Apogeo. Para este calculo se realizó una interpolación de la órbita, y por su simétria se tomó el tiempo de vuelo como el doble del tiempo que tomó llegar al punto deseado de la orbita lunar ($Q\simeq386\,000\,km$). Este tiempo de llegada a la Luna se calculó mediante método de raíces para conocer el tiempo en el que la nave cruzó por el eje $\hat{y}$ en su Apogeo. El perigeo ($q\simeq=8\,900\,km$), por la simetría se calculó como el punto al que llega la nave al final del tiempo de vuelo.

Aquí se establecen las condiciones iniciales de posición y velocidad de cada uno de los cuerpos. También se define la anomalía adecuada para la orbita de retorno ($f_0=227.5°$)

In [7]:
# @title
#Posicion y Velocidad inicial, Tierra
r0_earth = np.array([-alpha, 0, 0])
v0_earth = np.array([0, -alpha, 0])

#Posicion inicial, Luna
r0_moon = np.array([1-alpha, 0, 0])

#Anomalia adecuada
f = np.deg2rad(180 + 47.5)

#Posicion y Velocidad inicial Nave
r0 = r0_earth + np.array([R1*np.cos(f), R1*np.sin(f), 0])

v0 = np.array([vq*np.sin(f), -vq*np.cos(f), 0]) #Orbita Retrograda

Se realiza la transformación de las condiciones iniciales al sistema rotante.

In [8]:
# @title
#Condiciones iniciales del sistema rotante
r0_rot = r0.copy()
v0_rot = v0 - np.cross(omega_vec, r0)

Ejecutamos `crtbp_solucion` en intervalo de tiempo igual a $0.7678\,T_{Hohmann}$ pues se observó que para este lapso, la nave arrivó a aproximadamente su órbita de partida.

In [9]:
# @title
#Se escoge el factor 0.7678 por criterio de observación
ts = np.linspace(0, 0.7678*T_hohmann, 1000)

#Solucion CRTBP
solve = pc.crtbp_solucion(alpha, r0_rot, v0_rot, ts)
rs_rot, vs_rot, rs_ine, vs_ine, r1, r2 = solve

Realizamos la interpolación de la órbita para encontrar el tiempo en el que la nave alcanza el Apogeo. El Apogeo de la órbita se definió como el punto en el que la nave corta con el eje $\hat{y}$ debido a la simetría de la órbita. El tiempo de vuelo se calculó como el doble del tiempo en que alcanzó el Apogeo

In [10]:
# @title
#Interpoladores de la Orbita
interp_y = CubicHermiteSpline(ts, rs_rot[:,1], vs_rot[:,1])
interp_x = CubicHermiteSpline(ts, rs_rot[:,0], vs_rot[:,0])

#Tiempo de vuelo, Metodo de Raices con intervalo donde corta eje Y
T_flight = 2*ridder(interp_y, 0.8, 0.9)

#Apogeo y Perigeo de la Orbita
Q_orbit = np.array([interp_x(T_flight/2), interp_y(T_flight/2), 0])
q_orbit = np.array([interp_x(T_flight), interp_y(ts[-1]), 0])

#Apogeo y Perigeo Inercial
M = spy.rotate(omega*T_flight/2, 3)
Q_ine = Q_orbit @ M
q_ine = q_orbit @ M

Mostramos resultados obtenidos de la órbita

In [30]:
# @title
pd.DataFrame([np.rad2deg(f), T_flight*UT/86400,
              np.linalg.norm(Q_orbit)*UL, np.linalg.norm(q_orbit)*UL],
              index = ['f0 Adecuado (deg)', 'T Vuelo (day)',
                       'Apogeo (m)', 'Perigeo (m)'],
             columns = ['Algunos Valores'])

,Algunos Valores
f0 Adecuado (deg),2.275000e+02
T Vuelo (day),7.559220e+00
Apogeo (m),3.860738e+08
Perigeo (m),8.917551e+06


In [11]:
# @title
fig = go.Figure()

fig.update_layout(title = "Free Return Orbit (Rotating Reference Frame)",
                  width = 800, height = 600,
                  title_x = 0.5)

fig.add_trace(go.Scatter(
              x = [r0_earth[0]],
              y = [r0_earth[1]],
              mode = 'markers', name = 'Earth',
              line = {'color': 'blue'},
              marker = dict(size = 8)))

fig.add_trace(go.Scatter(
              x = [r0_moon[0]],
              y = [r0_moon[1]],
              mode = 'markers', name = 'Moon',
              line = {'color': 'grey'},
              marker = dict(size =6)))

fig.add_trace(go.Scatter(
              x = rs_rot[:, 0],
              y = rs_rot[:, 1],
              mode = 'markers', name = "Spaceship Orbit",
              line = {'color': 'green'},
              marker = dict(size =2)))

fig.add_trace(go.Scatter(
              x = [Q_orbit[0]],
              y = [Q_orbit[1]],
              mode = 'markers', name = 'Apogee',
              line = {'color': 'orange'},
              marker = dict(size =5)))

fig.add_trace(go.Scatter(
              x = [q_orbit[0]],
              y = [q_orbit[1]],
              mode = 'markers', name = 'Perigee',
              line = {'color': 'red'},
              marker = dict(size =5)))

fig.show()

In [12]:
# @title
fig = go.Figure()

fig.update_layout(title = "Free Return Orbit (Inertial Reference Frame)",
                  width = 800, height = 600,
                  title_x = 0.5)

fig.add_trace(go.Scatter(
              x = r1[:, 0],
              y = r1[:, 1],
              mode = 'markers', name = 'Earth',
              line = {'color': 'blue'},
              marker = dict(size = 3)))

fig.add_trace(go.Scatter(
              x = r2[:, 0],
              y = r2[:, 1],
              mode = 'markers', name = 'Moon',
              line = {'color': 'grey'},
              marker = dict(size =3)))

fig.add_trace(go.Scatter(
              x = rs_ine[:, 0],
              y = rs_ine[:, 1],
              mode = 'markers', name = "Spaceship Orbit",
              line = {'color': 'green'},
              marker = dict(size =2)))

fig.add_trace(go.Scatter(
              x = [Q_ine[0]],
              y = [Q_ine[1]],
              mode = 'markers', name = 'Apogee',
              line = {'color': 'orange'},
              marker = dict(size =5)))

fig.add_trace(go.Scatter(
              x = [q_ine[0]],
              y = [q_ine[1]],
              mode = 'markers', name = 'Perigee',
              line = {'color': 'red'},
              marker = dict(size =5)))



---



#**Constante de Jacobi y Regiones de Exclusión**

- **Sobre la Constante de Jacobi y las Regiones de Exclusión**

  Cuando se calculó la constante de Jacobi ($C_J\simeq1.41$) y se calcularon las regiones de exclusión, se identificó que no existían estas mismas y por tanto la nave tenía la posibilidad de escapar de la atracción de los dos cuerpos principales.

  En el gráfico de contorno se muestran los valores de $v$ que resultan siempre positivos. Además se anadieron los radios de las regiones cercanas a los dos cuerpos el radio central, y nos percatamos que la región permitida por el radio central se sobre pone con la región de exclusión del radio alrededor de la Tierra. Esto es lo que ocasiona que la región permitida se propague hacia el infinito donde la nave alcanzaría la superficie de velocidad cero.

Se calcula $C_J$

In [19]:
# @title
Cj = pc.constante_jacobi(alpha, rs_rot, vs_rot)[0]
pd.DataFrame([Cj], columns = ['Jacobi Constant'])

,Jacobi Constant
0,1.417535


Se calculan los contornos de las regiones

In [14]:
# @title
rango = 5

xr = np.linspace(-rango, rango, 100)
yr = xr.copy()

X,Y = np.meshgrid(xr, yr)

V = (2*(1-alpha)/((X+alpha)**2 + Y**2)**0.5 + 2*alpha/((X-(1-alpha))**2 + Y**2)**0.5 + X**2 + Y**2 - Cj)**0.5

Se calculan los radios de las regiones alrededor de los dos cuerpos y de la región central

In [15]:
# @title
theta = np.linspace(0, 2*np.pi, 360)

RCj_1 = 2*(1-alpha)/Cj * np.array([np.cos(theta), np.sin(theta)])
RCj_2 = 2*alpha/(Cj-(1-alpha)**2) * np.array([np.cos(theta), np.sin(theta)])
RCj = Cj**0.5 * np.array([np.cos(theta), np.sin(theta)])

In [16]:
# @title
fig = go.Figure()

fig.update_layout(title = "Free Return Orbit (Rotating Reference Frame)",
                  width = 800, height = 600,
                  title_x = 0.5)

fig.add_trace(go.Contour(
              z = V,
              x = xr,
              y = yr,
              colorscale = 'viridis',
              line_smoothing = 0.85,
              colorbar=dict(thickness=25,
                            thicknessmode = 'pixels',
                            len = 0.6,
                            lenmode = 'fraction',
                            outlinewidth = 0),
              contours = dict(start = 0, end = 10, size =1)))

fig.add_trace(go.Scatter(
              x = RCj_1[0] - alpha,
              y = RCj_1[1],
              mode = 'lines', name = 'R1',
              line = {'color': 'pink'},
              marker = dict(size = 2)))

fig.add_trace(go.Scatter(
              x = RCj_2[0] + 1 - alpha,
              y = RCj_2[1],
              mode = 'lines', name = 'R2',
              line = {'color': 'pink'},
              marker = dict(size = 2)))

fig.add_trace(go.Scatter(
              x = RCj[0],
              y = RCj[1],
              mode = 'lines', name = 'R',
              line = {'color': 'orange'},
              marker = dict(size = 2)))